In [1]:
import fiona
import numpy as np
import pandas as pd

In [2]:
filename = 'qgis_assets/modulos_automatico_final_new.shp'
shp = fiona.open(filename)

In [3]:
centroid_table = []
elems = []

for (idx, elem) in enumerate(shp):
    # Calculate centroid
    centroid = [0, 0]
    for coord in elem['geometry']['coordinates'][0]:
        centroid[0] += coord[0]
        centroid[1] += coord[1]
    centroid[0] /= len(elem['geometry']['coordinates'][0])
    centroid[1] /= len(elem['geometry']['coordinates'][0])

    centroid_table.append(np.asarray(centroid))
    elems.append(elem)

In [4]:
# Compare every centroid to check if they are the same
for (idx, elem) in enumerate(centroid_table):
    for (idx2, elem2) in enumerate(centroid_table):
        if idx != idx2:
            if np.linalg.norm(elem - elem2) < 0.0001:
                print('Centroid {} and {} are the same'.format(idx, idx2))

                # Remove the first one
                centroid_table.pop(idx)
                elems.pop(idx)
                idx -= 1
                break

In [5]:
# Write back
schema = shp.schema.copy()
input_crs = shp.crs

with fiona.open(filename.split('.')[0] + '_fixed_geometry.shp', 'w', 'ESRI Shapefile', schema, input_crs) as output:
    for (idx, elem) in enumerate(elems):
        output.write({'properties': elem['properties'], 'geometry': elem['geometry']})